In [8]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

In [9]:
def pad_tensor(tensor, target_shape):
    padding = [0]*4
    padding[1] = target_shape[1] - tensor.shape[1]
    padding[3] = target_shape[0] - tensor.shape[0]
    print(padding, tensor.shape)
    resul = F.pad(tensor, padding,"constant", 0)
    print(target_shape,resul.shape)
    return resul

In [10]:
def custom_collate_fn(batch):
    # print(batch[0])
    # 找到每个张量的最大形状
    max_shape = {}
    for key in batch[0].keys():
        if batch[0][key].dim() > 0:
            max_shape[key] = [max(item[key].shape[i] for item in batch) for i in range(2)]
            # print(max(item[key].shape[0] for item in batch),max(item[key].shape[1] for item in batch), key)    # 取道了最大的外形，没有问题
        # else:
            # print(key, batch[0][key].shape)

    # for key in max_shape.keys():
    #     print(key)
    #     print(max_shape[key])
    # 对每个样本的每个张量进行填充，并合并到一个批次中
    collated_batch = {}
    for key in batch[0].keys():
        if isinstance(batch[0][key], torch.Tensor) and batch[0][key].dim() > 0:
            # for item in batch:
            #     for keys in item.keys():
                    # print(keys, item[keys].shape)
            padded_tensors = []
            for item in batch:
                padded_tensor = pad_tensor(item[key], max_shape[key])
                # print(padded_tensor.shape, max_shape[key])
                padded_tensors.append(padded_tensor)
            # padded_tensors = [pad_tensor(item[key], max_shape[key]) for item in batch]
            # for item in batch:
            #     for keys in item.keys():
            #         pad_tensor(item[keys], max_shape[keys])
            #         print(max_shape[keys])         
            # for tensor in padded_tensors:
            #     print(key, tensor.shape)
            collated_batch[key] = torch.stack(padded_tensors, dim=0)
        else:
            collated_batch[key] = torch.stack([item[key] for item in batch])
    return collated_batch

    # for key in batch[0].keys():
    #     if key is 'label':
    #         max_shape[key] = len(batch[0][key])
    #     else:
    #         max_shape[key] = [max(item[key].shape[i] for item in batch) for i in range(len(batch[0][key].shape))]
    #
    # # 对每个样本的每个张量进行填充，并合并到一个批次中
    # collated_batch = {}
    # for key in batch[0].keys():
    #     if 'label' in key:
    #         collated_batch[key] = torch.stack([item[key] for item in batch])
    #     else:
    #         padded_tensors = [pad_tensor(item[key], max_shape[key]) for item in batch]
    #         collated_batch[key] = torch.stack(padded_tensors, dim=0)


In [11]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [12]:
data = [
    {
        'adj_tensor1': torch.randn(14, 77),
        'attr_tensor1': torch.randn(14, 80),
        'adj_tensor2': torch.randn(14, 77),
        'attr_tensor2': torch.randn(14, 80),
        # 'label': torch.tensor(1.0, dtype=torch.float32)
    },
    {
        'adj_tensor1': torch.randn(10, 81),
        'attr_tensor1': torch.randn(10, 85),
        'adj_tensor2': torch.randn(10, 81),
        'attr_tensor2': torch.randn(10, 85),
        # 'label': torch.tensor(0.0, dtype=torch.float32)
    },
    {
        'adj_tensor1': torch.randn(14, 81),
        'attr_tensor1': torch.randn(21, 85),
        'adj_tensor2': torch.randn(16, 81),
        'attr_tensor2': torch.randn(11, 85),
        # 'label': torch.tensor(0.0, dtype=torch.float32)
    }
]

In [13]:
dataset = CustomDataset(data)

In [14]:
# 使用自定义的 collate_fn 创建 DataLoader
dataloader = DataLoader(dataset, batch_size=3, collate_fn=custom_collate_fn)

# 打印批次数据
for batch in dataloader:
    print(batch)

[0, 4, 0, 0] torch.Size([14, 77])
[14, 81] torch.Size([14, 81])
[0, 0, 0, 4] torch.Size([10, 81])
[14, 81] torch.Size([14, 81])
[0, 0, 0, 0] torch.Size([14, 81])
[14, 81] torch.Size([14, 81])
[0, 5, 0, 7] torch.Size([14, 80])
[21, 85] torch.Size([21, 85])
[0, 0, 0, 11] torch.Size([10, 85])
[21, 85] torch.Size([21, 85])
[0, 0, 0, 0] torch.Size([21, 85])
[21, 85] torch.Size([21, 85])
[0, 4, 0, 2] torch.Size([14, 77])
[16, 81] torch.Size([16, 81])
[0, 0, 0, 6] torch.Size([10, 81])
[16, 81] torch.Size([16, 81])
[0, 0, 0, 0] torch.Size([16, 81])
[16, 81] torch.Size([16, 81])
[0, 5, 0, 0] torch.Size([14, 80])
[14, 85] torch.Size([14, 85])
[0, 0, 0, 4] torch.Size([10, 85])
[14, 85] torch.Size([14, 85])
[0, 0, 0, 3] torch.Size([11, 85])
[14, 85] torch.Size([14, 85])
{'adj_tensor1': tensor([[[-1.3173, -0.7021,  0.0049,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.9084,  0.5251,  0.6145,  ...,  0.0000,  0.0000,  0.0000],
         [-1.1691, -1.4345, -0.0865,  ...,  0.0000,  0.0000,  0.0000],
  

In [ ]:
a = torch.ones((14, 77), dtype=torch.float32)
test = pad_tensor(a, (21, 85))

In [ ]:
test.shape